In [1]:
#Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
import copy
import warnings
# import math
# from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import f1_score,accuracy_score,recall_score,precision_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, auc
# from sklearn.preprocessing import label_binarize
# from scipy import interp
# from itertools import cycle, zip_longest, chain
# import pickle
# from MVDD.MVDD import MVDD
# import MVDD.MVDD_Generator as mvGen

import model_comp as MC

%matplotlib inline
%load_ext autoreload
%autoreload 2

plt.rc('font', size=12)
warnings.filterwarnings('ignore')

In [2]:
## Load Hemo Training Escape Data
#Load original dataframes
escHemo = pd.read_csv('../Data/Preprocessed Data/ESCAPE_Hemo.csv', index_col='ID')
escHemodeath = escHemo['ScoreDeath']
escHemorehosp = escHemo['ScoreRehosp']
escHemoreadm = escHemo['ScoreReadmission']

# Preprocess and create training and testing sets
escHemo = escHemo.drop('Score', axis=1)
escHemo = escHemo.drop('ScoreDeath', axis=1)
escHemo = escHemo.drop('ScoreRehosp', axis=1)
escHemo = escHemo.drop('ScoreReadmission', axis=1)
escHemo = escHemo.replace(np.inf, 0)
escHemo = escHemo.fillna(0)

In [3]:
## Load hemo validation
# card shock
cardShockHemo = pd.read_csv("../Data Validation/Cardiogenic Shock/Preprocessed Data/CardiogenicShock_Hemo.csv", sep=",", index_col='ID')
cardShockHemodeath = cardShockHemo['ScoreDeath']
cardShockHemorehosp = cardShockHemo['ScoreRehosp']

cardShockHemo = cardShockHemo.drop('ScoreDeath', axis=1)
cardShockHemo = cardShockHemo.drop('Score', axis=1)
cardShockHemo = cardShockHemo.drop('ScoreRehosp', axis=1)
cardShockHemo = cardShockHemo.drop('ScoreReadmission', axis=1)
cardShockHemo = cardShockHemo.replace(np.inf, 0)
cardShockHemo = cardShockHemo.fillna(0)

# serial cardiac
serialHemo = pd.read_csv("../Data Validation/Serial Cardiac Caths/Preprocessed Data/SerialCardiac_Hemo.csv", sep=",", index_col='ID')
serialHemodeath = serialHemo['ScoreDeath']
serialHemorehosp = serialHemo['ScoreRehosp']

serialHemo = serialHemo.drop('ScoreDeath', axis=1)
serialHemo = serialHemo.drop('Score', axis=1)
serialHemo = serialHemo.drop('ScoreRehosp', axis=1)
serialHemo = serialHemo.drop('ScoreReadmission', axis=1)
serialHemo = serialHemo.replace(np.inf, 0)
serialHemo = serialHemo.fillna(0)



In [4]:
## Load all data Training Escape Data
#Load original dataframes
escAllData = pd.read_csv('../Data/Preprocessed Data/ESCAPE_AllData.csv', index_col='ID')
escAllDatadeath = escAllData['ScoreDeath']
escAllDatarehosp = escAllData['ScoreRehosp']
escAllDatareadm = escAllData['ScoreReadmission']

# Preprocess and create training and testing sets
escAllData = escAllData.drop('Score', axis=1)
escAllData = escAllData.drop('ScoreDeath', axis=1)
escAllData = escAllData.drop('ScoreRehosp', axis=1)
escAllData = escAllData.drop('ScoreReadmission', axis=1)
escAllData = escAllData.replace(np.inf, 0)
escAllData = escAllData.fillna(0)


In [5]:
## Load all data validation
# card shock
cardShockAllData = pd.read_csv("../Data Validation/Cardiogenic Shock/Preprocessed Data/CardiogenicShock_AllData.csv", sep=",", index_col='ID')
cardShockAllDatadeath = cardShockAllData['ScoreDeath']
cardShockAllDatarehosp = cardShockAllData['ScoreRehosp']

cardShockAllData = cardShockAllData.drop('ScoreDeath', axis=1)
cardShockAllData = cardShockAllData.drop('Score', axis=1)
cardShockAllData = cardShockAllData.drop('ScoreRehosp', axis=1)
cardShockAllData = cardShockAllData.drop('ScoreReadmission', axis=1)
cardShockAllData = cardShockAllData.replace(np.inf, 0)
cardShockAllData = cardShockAllData.fillna(0)

# serial cardiac
serialAllData = pd.read_csv("../Data Validation/Serial Cardiac Caths/Preprocessed Data/SerialCardiac_AllData.csv", sep=",", index_col='ID')
serialAllDatadeath = serialAllData['ScoreDeath']
serialAllDatarehosp = serialAllData['ScoreRehosp']

serialAllData = serialAllData.drop('ScoreDeath', axis=1)
serialAllData = serialAllData.drop('Score', axis=1)
serialAllData = serialAllData.drop('ScoreRehosp', axis=1)
serialAllData = serialAllData.drop('ScoreReadmission', axis=1)
serialAllData = serialAllData.replace(np.inf, 0)
serialAllData= serialAllData.fillna(0)

#best
bestAllData = pd.read_csv("../Data Validation/BEST/Preprocessed Data/BEST_AllData.csv", sep=",", index_col='ID')
bestAllDatadeath = bestAllData['ScoreDeath']
bestAllDatarehosp = bestAllData['ScoreRehosp']

bestAllData = bestAllData.drop('ScoreDeath', axis=1)
bestAllData = bestAllData.drop('Score', axis=1)
bestAllData = bestAllData.drop('ScoreRehosp', axis=1)
bestAllData = bestAllData.drop('ScoreReadmission', axis=1)
bestAllData = bestAllData.replace(np.inf, 0)
bestAllData  = bestAllData.fillna(0)

#guide
guideAllData = pd.read_csv("../Data Validation/GUIDE-IT/Preprocessed Data/GUIDE-IT_AllData.csv", sep=",", index_col='ID')
guideAllDatadeath = guideAllData['ScoreDeath']
guideAllDatarehosp = guideAllData['ScoreDeath']

guideAllData = guideAllData.drop('ScoreDeath', axis=1)
guideAllData = guideAllData.drop('Score', axis=1)
guideAllData = guideAllData.drop('ScoreRehosp', axis=1)
guideAllData = guideAllData.drop('ScoreReadmission', axis=1)
guideAllData = guideAllData.replace(np.inf, 0)
guideAllData = guideAllData.fillna(0)

# Hemodynamics

## DeLvTx Outcome

In [6]:
xData = copy.deepcopy(escHemo)
yData = escHemodeath
xData.shape, yData.shape

# KNN
modelName = "KNN"
knn = KNeighborsClassifier()

knn, knn_train_mets = MC.trainCrossVal(xData, yData, model=knn, numFolds=5, modelName=modelName, showIndividualROC=False)
cs_knn_mets = MC.performValidation(knn, cardShockHemo, cardShockHemodeath, modelName)
ser_knn_mets = MC.performValidation(knn, serialHemo, serialHemodeath, modelName)

# DT
modelName = "DT"
dt = DecisionTreeClassifier(random_state=100, max_depth=None)

dt, dt_train_mets = MC.trainCrossVal(xData, yData, model=dt, numFolds=5, modelName=modelName, showIndividualROC=False)
cs_dt_mets = MC.performValidation(dt, cardShockHemo, cardShockHemodeath, modelName)
ser_dt_mets = MC.performValidation(dt, serialHemo, serialHemodeath, modelName)

# RF
modelName = "RF"
rf = RandomForestClassifier(random_state=100, n_estimators=1000)

rf, rf_train_mets = MC.trainCrossVal(xData, yData, model=rf, numFolds=5, modelName=modelName, showIndividualROC=False)
cs_rf_mets = MC.performValidation(rf, cardShockHemo, cardShockHemodeath, modelName)
ser_rf_mets = MC.performValidation(rf, serialHemo, serialHemodeath, modelName)


metLst = [knn_train_mets, cs_knn_mets, ser_knn_mets,
          dt_train_mets, cs_dt_mets, ser_dt_mets, 
          rf_train_mets, cs_rf_mets, ser_rf_mets]

# Make dataframe of stats
columns=["Model", "TPR", "TPR SD", "TPR CI", "TNR", "TNR SD", "TNR CI", "PPV", "PPV SD", "PPV CI",
             "NPV", "NPV SD", "NPV CI", "FPR", "FPR SD", "FPR CI", "FNR", "FNR SD", "FNR CI",
             "Acc", "Acc SD", "Acc CI", "AUC", "AUC SD", "AUC CI"]

mets = pd.DataFrame(metLst, columns=columns)
mets.index = ["Escape", "Card Shock", "Serial", "Escape", "Card Shock", "Serial", "Escape", "Card Shock", "Serial"]
mets.to_csv("Hemo_DeLvTx_Results.csv")
mets

,Model,TPR,TPR SD,TPR CI,TNR,TNR SD,TNR CI,PPV,PPV SD,PPV CI,...,FPR CI,FNR,FNR SD,FNR CI,Acc,Acc SD,Acc CI,AUC,AUC SD,AUC CI
Escape,KNN,0.253810,0.308119,0.067269,0.810508,0.292851,0.075547,0.291974,0.394758,0.061836,...,0.075547,0.746190,0.308119,0.067269,0.726254,0.294094,0.064488,0.532159,0.097572,0.024313
Card Shock,KNN,0.153669,0.248209,0.093867,0.789186,0.408143,0.085774,0.217546,0.215152,0.084770,...,0.085774,0.846331,0.248209,0.093867,0.659603,0.271423,0.063722,0.471428,0.083983,0.026961
Serial,KNN,0.198932,0.353112,0.064501,0.801329,0.381967,0.064529,0.239986,0.207662,0.059942,...,0.064529,0.801068,0.353112,0.064501,0.684892,0.232936,0.050547,0.500130,0.038496,0.001342
Escape,DT,0.476962,0.522776,0.020578,0.868067,0.204225,0.082252,0.482977,0.544813,0.017689,...,0.082252,0.523038,0.522776,0.020578,0.805783,0.248658,0.074970,0.672514,0.303405,0.056311
Card Shock,DT,0.405147,0.297197,0.049124,0.850047,0.162846,0.094369,0.411314,0.164414,0.047500,...,0.094369,0.594853,0.297197,0.049124,0.758940,0.087297,0.081165,0.627597,0.090809,0.056975
Serial,DT,0.438461,0.272589,0.029161,0.859872,0.093251,0.070519,0.432859,0.215596,0.030460,...,0.070519,0.561539,0.272589,0.029161,0.776259,0.083205,0.061786,0.649167,0.120704,0.045401
Escape,RF,0.431250,0.772071,0.035548,0.877062,0.303589,0.083251,0.455735,0.593814,0.028524,...,0.083251,0.568750,0.772071,0.035548,0.831566,0.225654,0.078067,0.654156,0.337273,0.053231
Card Shock,RF,0.425769,0.817648,0.043457,0.849047,0.354039,0.094234,0.480770,0.261939,0.022119,...,0.094234,0.574231,0.817648,0.043457,0.753642,0.145189,0.080330,0.637408,0.261364,0.059125
Serial,RF,0.400481,0.836170,0.037084,0.854784,0.316660,0.070019,0.415873,0.347128,0.034096,...,0.070019,0.599519,0.836170,0.037084,0.771942,0.136622,0.061302,0.627633,0.282552,0.041997


## Rehosp Outcome

In [7]:
xData = copy.deepcopy(escHemo)
yData = escHemorehosp
xData.shape, yData.shape

# KNN
modelName = "KNN"
knn = KNeighborsClassifier()

knn, knn_train_mets = MC.trainCrossVal(xData, yData, model=knn, numFolds=5, modelName=modelName, showIndividualROC=False)
cs_knn_mets = MC.performValidation(knn, cardShockHemo, cardShockHemorehosp, modelName)
ser_knn_mets = MC.performValidation(knn, serialHemo, serialHemorehosp, modelName)

# DT
modelName = "DT"
dt = DecisionTreeClassifier(random_state=100, max_depth=None)

dt, dt_train_mets = MC.trainCrossVal(xData, yData, model=dt, numFolds=5, modelName=modelName, showIndividualROC=False)
cs_dt_mets = MC.performValidation(dt, cardShockHemo, cardShockHemorehosp, modelName)
ser_dt_mets = MC.performValidation(dt, serialHemo, serialHemorehosp, modelName)

# RF
modelName = "RF"
rf = RandomForestClassifier(random_state=100, n_estimators=1000)

rf, rf_train_mets = MC.trainCrossVal(xData, yData, model=rf, numFolds=5, modelName=modelName, showIndividualROC=False)
cs_rf_mets = MC.performValidation(rf, cardShockHemo, cardShockHemorehosp, modelName)
ser_rf_mets = MC.performValidation(rf, serialHemo, serialHemorehosp, modelName)


metLst = [knn_train_mets, cs_knn_mets, ser_knn_mets,
          dt_train_mets, cs_dt_mets, ser_dt_mets, 
          rf_train_mets, cs_rf_mets, ser_rf_mets]

# Make dataframe of stats
columns=["Model", "TPR", "TPR SD", "TPR CI", "TNR", "TNR SD", "TNR CI", "PPV", "PPV SD", "PPV CI",
             "NPV", "NPV SD", "NPV CI", "FPR", "FPR SD", "FPR CI", "FNR", "FNR SD", "FNR CI",
             "Acc", "Acc SD", "Acc CI", "AUC", "AUC SD", "AUC CI"]

mets = pd.DataFrame(metLst, columns=columns)
mets.index = ["Escape", "Card Shock", "Serial", "Escape", "Card Shock", "Serial", "Escape", "Card Shock", "Serial"]
mets.to_csv("Hemo_Rehosp_Results.csv")
mets

,Model,TPR,TPR SD,TPR CI,TNR,TNR SD,TNR CI,PPV,PPV SD,PPV CI,...,FPR CI,FNR,FNR SD,FNR CI,Acc,Acc SD,Acc CI,AUC,AUC SD,AUC CI
Escape,KNN,0.253810,0.308119,0.067269,0.810508,0.292851,0.075547,0.291974,0.394758,0.061836,...,0.075547,0.746190,0.308119,0.067269,0.726254,0.294094,0.064488,0.532159,0.097572,0.024313
Card Shock,KNN,0.153669,0.248209,0.093867,0.789186,0.408143,0.085774,0.217546,0.215152,0.084770,...,0.085774,0.846331,0.248209,0.093867,0.659603,0.271423,0.063722,0.471428,0.083983,0.026961
Serial,KNN,0.198932,0.353112,0.064501,0.801329,0.381967,0.064529,0.239986,0.207662,0.059942,...,0.064529,0.801068,0.353112,0.064501,0.684892,0.232936,0.050547,0.500130,0.038496,0.001342
Escape,DT,0.476962,0.522776,0.020578,0.868067,0.204225,0.082252,0.482977,0.544813,0.017689,...,0.082252,0.523038,0.522776,0.020578,0.805783,0.248658,0.074970,0.672514,0.303405,0.056311
Card Shock,DT,0.405147,0.297197,0.049124,0.850047,0.162846,0.094369,0.411314,0.164414,0.047500,...,0.094369,0.594853,0.297197,0.049124,0.758940,0.087297,0.081165,0.627597,0.090809,0.056975
Serial,DT,0.438461,0.272589,0.029161,0.859872,0.093251,0.070519,0.432859,0.215596,0.030460,...,0.070519,0.561539,0.272589,0.029161,0.776259,0.083205,0.061786,0.649167,0.120704,0.045401
Escape,RF,0.431250,0.772071,0.035548,0.877062,0.303589,0.083251,0.455735,0.593814,0.028524,...,0.083251,0.568750,0.772071,0.035548,0.831566,0.225654,0.078067,0.654156,0.337273,0.053231
Card Shock,RF,0.425769,0.817648,0.043457,0.849047,0.354039,0.094234,0.480770,0.261939,0.022119,...,0.094234,0.574231,0.817648,0.043457,0.753642,0.145189,0.080330,0.637408,0.261364,0.059125
Serial,RF,0.400481,0.836170,0.037084,0.854784,0.316660,0.070019,0.415873,0.347128,0.034096,...,0.070019,0.599519,0.836170,0.037084,0.771942,0.136622,0.061302,0.627633,0.282552,0.041997


## Readmission Outcome

In [8]:
xData = copy.deepcopy(escHemo)
yData = escHemoreadm
xData.shape, yData.shape

# KNN
modelName = "KNN"
knn = KNeighborsClassifier()
knn, knn_train_mets = MC.trainCrossVal(xData, yData, model=knn, numFolds=5, modelName=modelName, showIndividualROC=False)

# DT
modelName = "DT"
dt = DecisionTreeClassifier(random_state=100, max_depth=None)
dt, dt_train_mets = MC.trainCrossVal(xData, yData, model=dt, numFolds=5, modelName=modelName, showIndividualROC=False)

# RF
modelName = "RF"
rf = RandomForestClassifier(random_state=100, n_estimators=1000)
rf, rf_train_mets = MC.trainCrossVal(xData, yData, model=rf, numFolds=5, modelName=modelName, showIndividualROC=False)


metLst = [knn_train_mets, dt_train_mets, rf_train_mets]

# Make dataframe of stats
columns=["Model", "TPR", "TPR SD", "TPR CI", "TNR", "TNR SD", "TNR CI", "PPV", "PPV SD", "PPV CI",
             "NPV", "NPV SD", "NPV CI", "FPR", "FPR SD", "FPR CI", "FNR", "FNR SD", "FNR CI",
             "Acc", "Acc SD", "Acc CI", "AUC", "AUC SD", "AUC CI"]

mets = pd.DataFrame(metLst, columns=columns)
mets.index = ["Escape",  "Escape", "Escape"]
mets.to_csv("Hemo_Readm_Results.csv")
mets

,Model,TPR,TPR SD,TPR CI,TNR,TNR SD,TNR CI,PPV,PPV SD,PPV CI,...,FPR CI,FNR,FNR SD,FNR CI,Acc,Acc SD,Acc CI,AUC,AUC SD,AUC CI
Escape,KNN,0.253810,0.308119,0.067269,0.810508,0.292851,0.075547,0.291974,0.394758,0.061836,...,0.075547,0.746190,0.308119,0.067269,0.726254,0.294094,0.064488,0.532159,0.097572,0.024313
Escape,DT,0.476962,0.522776,0.020578,0.868067,0.204225,0.082252,0.482977,0.544813,0.017689,...,0.082252,0.523038,0.522776,0.020578,0.805783,0.248658,0.074970,0.672514,0.303405,0.056311
Escape,RF,0.431250,0.772071,0.035548,0.877062,0.303589,0.083251,0.455735,0.593814,0.028524,...,0.083251,0.568750,0.772071,0.035548,0.831566,0.225654,0.078067,0.654156,0.337273,0.053231


# All Data

## DeLvTx Outcome

In [9]:
xData = copy.deepcopy(escAllData)
yData = escAllDatadeath
xData.shape, yData.shape

# KNN
modelName = "KNN"
knn = KNeighborsClassifier()

knn, knn_train_mets = MC.trainCrossVal(xData, yData, model=knn, numFolds=5, modelName=modelName, showIndividualROC=False)
cs_knn_mets = MC.performValidation(knn, cardShockAllData, cardShockAllDatadeath, modelName)
ser_knn_mets = MC.performValidation(knn, serialAllData, serialAllDatadeath, modelName)
best_knn_mets = MC.performValidation(knn, bestAllData, bestAllDatadeath, modelName)
guide_knn_mets = MC.performValidation(knn, guideAllData, guideAllDatadeath, modelName)


# DT
modelName = "DT"
dt = DecisionTreeClassifier(random_state=100, max_depth=None)

dt, dt_train_mets = MC.trainCrossVal(xData, yData, model=dt, numFolds=5, modelName=modelName, showIndividualROC=False)
cs_dt_mets = MC.performValidation(dt, cardShockAllData, cardShockAllDatadeath, modelName)
ser_dt_mets = MC.performValidation(dt, serialAllData, serialAllDatadeath, modelName)
best_dt_mets = MC.performValidation(dt, bestAllData, bestAllDatadeath, modelName)
guide_dt_mets = MC.performValidation(dt, guideAllData, guideAllDatadeath, modelName)

# RF
modelName = "RF"
rf = RandomForestClassifier(random_state=100, n_estimators=1000)

rf, rf_train_mets = MC.trainCrossVal(xData, yData, model=rf, numFolds=5, modelName=modelName, showIndividualROC=False)
cs_rf_mets = MC.performValidation(rf, cardShockAllData, cardShockAllDatadeath, modelName)
ser_rf_mets = MC.performValidation(rf, serialAllData, serialAllDatadeath, modelName)
best_rf_mets = MC.performValidation(rf, bestAllData, bestAllDatadeath, modelName)
guide_rf_mets = MC.performValidation(rf, guideAllData, guideAllDatadeath, modelName)


metLst = [knn_train_mets, cs_knn_mets, ser_knn_mets, best_knn_mets, guide_knn_mets,
          dt_train_mets, cs_dt_mets, ser_dt_mets, best_dt_mets, guide_dt_mets,
          rf_train_mets, cs_rf_mets, ser_rf_mets, best_rf_mets, guide_rf_mets]

# Make dataframe of stats
columns=["Model", "TPR", "TPR SD", "TPR CI", "TNR", "TNR SD", "TNR CI", "PPV", "PPV SD", "PPV CI",
             "NPV", "NPV SD", "NPV CI", "FPR", "FPR SD", "FPR CI", "FNR", "FNR SD", "FNR CI",
             "Acc", "Acc SD", "Acc CI", "AUC", "AUC SD", "AUC CI"]

mets = pd.DataFrame(metLst, columns=columns)
mets.index = ["Escape", "Card Shock", "Serial", "Best", "Guide", 
              "Escape", "Card Shock", "Serial", "Best", "Guide",
              "Escape", "Card Shock", "Serial", "Best", "Guide"]
mets.to_csv("AllData_DeLvTx_Results.csv")
mets

,Model,TPR,TPR SD,TPR CI,TNR,TNR SD,TNR CI,PPV,PPV SD,PPV CI,...,FPR CI,FNR,FNR SD,FNR CI,Acc,Acc SD,Acc CI,AUC,AUC SD,AUC CI
Escape,KNN,0.689369,0.727618,0.040501,0.951287,0.239252,0.062522,0.787347,0.607394,0.049890,...,0.062522,0.310631,0.727618,0.040501,0.992784,0.010348,0.065334,0.815688,0.337517,0.052292
Card Shock,KNN,0.041543,0.083086,0.055224,0.624335,0.536365,0.028759,0.030973,0.061947,0.055857,...,0.028759,0.958457,0.083086,0.055224,0.349495,0.084338,0.031641,0.391187,0.217626,0.026904
Serial,KNN,0.021661,0.043321,0.077493,0.611503,0.618386,0.037414,0.040268,0.080537,0.075971,...,0.037414,0.978339,0.043321,0.077493,0.346405,0.170833,0.043912,0.408592,0.182817,0.033876
Best,KNN,0.525162,0.713063,0.008356,0.829649,0.520134,0.030244,0.616293,0.536834,0.017963,...,0.030244,0.474838,0.713063,0.008356,0.822463,0.391231,0.029912,0.677406,0.347088,0.022187
Guide,KNN,0.294736,0.703280,0.044680,0.854370,0.534335,0.058706,0.403007,0.775754,0.030713,...,0.058706,0.705264,0.703280,0.044680,0.849114,0.317979,0.058269,0.574553,0.126999,0.026927
Escape,DT,0.776115,0.713171,0.048905,0.971057,0.179716,0.063877,0.792639,0.601405,0.050347,...,0.063877,0.223885,0.713171,0.048905,0.993680,0.012908,0.065393,0.870829,0.349370,0.056676
Card Shock,DT,0.135015,0.270030,0.049274,0.596384,0.851877,0.025321,0.050388,0.100775,0.054689,...,0.025321,0.864985,0.270030,0.049274,0.385859,0.585109,0.027555,0.319341,0.361318,0.034666
Serial,DT,0.207581,0.415162,0.060590,0.595996,0.729315,0.034715,0.139903,0.279805,0.067237,...,0.034715,0.792419,0.415162,0.060590,0.458606,0.342980,0.022796,0.382895,0.234210,0.038343
Best,DT,0.469000,0.722308,0.009274,0.834589,0.484145,0.030470,0.645760,0.447447,0.020111,...,0.030470,0.531000,0.722308,0.009274,0.831130,0.387758,0.030312,0.651795,0.333536,0.020523
Guide,DT,0.277491,0.664341,0.046519,0.865595,0.449274,0.059629,0.331268,0.780083,0.040509,...,0.059629,0.722509,0.664341,0.046519,0.851646,0.280999,0.058480,0.571543,0.172435,0.026378


## Rehosp Outcome

In [10]:
xData = copy.deepcopy(escAllData)
yData = escAllDatarehosp
xData.shape, yData.shape

# KNN
modelName = "KNN"
knn = KNeighborsClassifier()

knn, knn_train_mets = MC.trainCrossVal(xData, yData, model=knn, numFolds=5, modelName=modelName, showIndividualROC=False)
cs_knn_mets = MC.performValidation(knn, cardShockAllData, cardShockAllDatarehosp, modelName)
ser_knn_mets = MC.performValidation(knn, serialAllData, serialAllDatarehosp, modelName)
best_knn_mets = MC.performValidation(knn, bestAllData, bestAllDatarehosp, modelName)
guide_knn_mets = MC.performValidation(knn, guideAllData, guideAllDatarehosp, modelName)


# DT
modelName = "DT"
dt = DecisionTreeClassifier(random_state=100, max_depth=None)

dt, dt_train_mets = MC.trainCrossVal(xData, yData, model=dt, numFolds=5, modelName=modelName, showIndividualROC=False)
cs_dt_mets = MC.performValidation(dt, cardShockAllData, cardShockAllDatarehosp, modelName)
ser_dt_mets = MC.performValidation(dt, serialAllData, serialAllDatarehosp, modelName)
best_dt_mets = MC.performValidation(dt, bestAllData, bestAllDatarehosp, modelName)
guide_dt_mets = MC.performValidation(dt, guideAllData, guideAllDatarehosp, modelName)

# RF
modelName = "RF"
rf = RandomForestClassifier(random_state=100, n_estimators=1000)

rf, rf_train_mets = MC.trainCrossVal(xData, yData, model=rf, numFolds=5, modelName=modelName, showIndividualROC=False)
cs_rf_mets = MC.performValidation(rf, cardShockAllData, cardShockAllDatarehosp, modelName)
ser_rf_mets = MC.performValidation(rf, serialAllData, serialAllDatarehosp, modelName)
best_rf_mets = MC.performValidation(rf, bestAllData, bestAllDatarehosp, modelName)
guide_rf_mets = MC.performValidation(rf, guideAllData, guideAllDatarehosp, modelName)


metLst = [knn_train_mets, cs_knn_mets, ser_knn_mets, best_knn_mets, guide_knn_mets,
          dt_train_mets, cs_dt_mets, ser_dt_mets, best_dt_mets, guide_dt_mets,
          rf_train_mets, cs_rf_mets, ser_rf_mets, best_rf_mets, guide_rf_mets]

# Make dataframe of stats
columns=["Model", "TPR", "TPR SD", "TPR CI", "TNR", "TNR SD", "TNR CI", "PPV", "PPV SD", "PPV CI",
             "NPV", "NPV SD", "NPV CI", "FPR", "FPR SD", "FPR CI", "FNR", "FNR SD", "FNR CI",
             "Acc", "Acc SD", "Acc CI", "AUC", "AUC SD", "AUC CI"]

mets = pd.DataFrame(metLst, columns=columns)
mets.index = ["Escape", "Card Shock", "Serial", "Best", "Guide", 
              "Escape", "Card Shock", "Serial", "Best", "Guide",
              "Escape", "Card Shock", "Serial", "Best", "Guide"]
mets.to_csv("AllData_Rehosp_Results.csv")
mets

,Model,TPR,TPR SD,TPR CI,TNR,TNR SD,TNR CI,PPV,PPV SD,PPV CI,...,FPR CI,FNR,FNR SD,FNR CI,Acc,Acc SD,Acc CI,AUC,AUC SD,AUC CI
Escape,KNN,0.689369,0.727618,0.040501,0.951287,0.239252,0.062522,0.787347,0.607394,0.049890,...,0.062522,0.310631,0.727618,0.040501,0.992784,0.010348,0.065334,0.815688,0.337517,0.052292
Card Shock,KNN,0.041543,0.083086,0.055224,0.624335,0.536365,0.028759,0.030973,0.061947,0.055857,...,0.028759,0.958457,0.083086,0.055224,0.349495,0.084338,0.031641,0.391187,0.217626,0.026904
Serial,KNN,0.021661,0.043321,0.077493,0.611503,0.618386,0.037414,0.040268,0.080537,0.075971,...,0.037414,0.978339,0.043321,0.077493,0.346405,0.170833,0.043912,0.408592,0.182817,0.033876
Best,KNN,0.525162,0.713063,0.008356,0.829649,0.520134,0.030244,0.616293,0.536834,0.017963,...,0.030244,0.474838,0.713063,0.008356,0.822463,0.391231,0.029912,0.677406,0.347088,0.022187
Guide,KNN,0.488889,0.826714,0.010395,0.802999,0.632041,0.054285,0.113402,0.340456,0.061318,...,0.054285,0.511111,0.826714,0.010395,0.611139,0.563964,0.032877,0.645944,0.403391,0.037675
Escape,DT,0.776115,0.713171,0.048905,0.971057,0.179716,0.063877,0.792639,0.601405,0.050347,...,0.063877,0.223885,0.713171,0.048905,0.993680,0.012908,0.065393,0.870829,0.349370,0.056676
Card Shock,DT,0.135015,0.270030,0.049274,0.596384,0.851877,0.025321,0.050388,0.100775,0.054689,...,0.025321,0.864985,0.270030,0.049274,0.385859,0.585109,0.027555,0.319341,0.361318,0.034666
Serial,DT,0.207581,0.415162,0.060590,0.595996,0.729315,0.034715,0.139903,0.279805,0.067237,...,0.034715,0.792419,0.415162,0.060590,0.458606,0.342980,0.022796,0.382895,0.234210,0.038343
Best,DT,0.469000,0.722308,0.009274,0.834589,0.484145,0.030470,0.645760,0.447447,0.020111,...,0.030470,0.531000,0.722308,0.009274,0.831130,0.387758,0.030312,0.651795,0.333536,0.020523
Guide,DT,0.369216,0.795350,0.035664,0.794129,0.543322,0.053484,0.391080,0.927872,0.032547,...,0.053484,0.630784,0.795350,0.035664,0.611646,0.513013,0.032952,0.581673,0.419374,0.028184


## Readm Outcome

In [11]:
xData = copy.deepcopy(escAllData)
yData = escAllDatareadm
xData.shape, yData.shape

# KNN
modelName = "KNN"
knn = KNeighborsClassifier()

knn, knn_train_mets = MC.trainCrossVal(xData, yData, model=knn, numFolds=5, modelName=modelName, showIndividualROC=False)

# DT
modelName = "DT"
dt = DecisionTreeClassifier(random_state=100, max_depth=None)

dt, dt_train_mets = MC.trainCrossVal(xData, yData, model=dt, numFolds=5, modelName=modelName, showIndividualROC=False)


# RF
modelName = "RF"
rf = RandomForestClassifier(random_state=100, n_estimators=1000)

rf, rf_train_mets = MC.trainCrossVal(xData, yData, model=rf, numFolds=5, modelName=modelName, showIndividualROC=False)


metLst = [knn_train_mets, dt_train_mets, rf_train_mets]

# Make dataframe of stats
columns=["Model", "TPR", "TPR SD", "TPR CI", "TNR", "TNR SD", "TNR CI", "PPV", "PPV SD", "PPV CI",
             "NPV", "NPV SD", "NPV CI", "FPR", "FPR SD", "FPR CI", "FNR", "FNR SD", "FNR CI",
             "Acc", "Acc SD", "Acc CI", "AUC", "AUC SD", "AUC CI"]

mets = pd.DataFrame(metLst, columns=columns)
mets.index = ["Escape", 
              "Escape", 
              "Escape"]
mets.to_csv("AllData_Readm_Results.csv")
mets

,Model,TPR,TPR SD,TPR CI,TNR,TNR SD,TNR CI,PPV,PPV SD,PPV CI,...,FPR CI,FNR,FNR SD,FNR CI,Acc,Acc SD,Acc CI,AUC,AUC SD,AUC CI
Escape,KNN,0.689369,0.727618,0.040501,0.951287,0.239252,0.062522,0.787347,0.607394,0.049890,...,0.062522,0.310631,0.727618,0.040501,0.992784,0.010348,0.065334,0.815688,0.337517,0.052292
Escape,DT,0.776115,0.713171,0.048905,0.971057,0.179716,0.063877,0.792639,0.601405,0.050347,...,0.063877,0.223885,0.713171,0.048905,0.993680,0.012908,0.065393,0.870829,0.349370,0.056676
Escape,RF,0.602896,0.825476,0.029854,0.943737,0.245869,0.061997,0.801207,0.593173,0.051079,...,0.061997,0.397104,0.825476,0.029854,0.993230,0.010105,0.065363,0.767958,0.368377,0.048177


In [ ]:
# TODO - need to figure out why across different outcome labels a lot of these are the same value ... 
# Potentially try drop NA instead of impute with zeros and infs

# Print out dataframes and compare btw outcomes